# MetaboLights to CMAP
## Krista Longnecker, 9 July 2025


MetaboLights has FTP access to their data files and that is easy enough to access, but there are some downstream steps to add because I did not upload the full station information to MetaboLights.

In [173]:
%reset -f

In [174]:
import pandas as pd
import os
import io
from ftplib import FTP
import re
from datetime import datetime, timedelta, timezone

#import libchebipy #pip install libChEBIpy
from libchebipy._chebi_entity import ChebiEntity #this will get metabolite synonyms

# import pdb
#pdb.set_trace()

In [175]:
#make the data folder if it is not already there (it is in .gitignore, so it will not end up at GitHub)
folder = "data"
os.chdir(".")

if os.path.isdir(folder):
    print("Data will go here (but should not be synced to GitHub): %s" % (os.getcwd()) + '\\' + folder)
else:
    os.mkdir(folder)

Data will go here (but should not be synced to GitHub): C:\Users\klongnecker\Documents\Dropbox\GitHub_espresso\data_pipeline\CMAP\data


### Get MetaboLights files via FTP access

The following cells read a file, write to disk, and then read the result into Python. I am not sure how to skip the middle/write step, but it seems like setting this up should work (but it doesn't)\
    # Create an in-memory binary stream\
    in_memory_file = io.BytesIO()

Note: the frictionless website talks about FTP access to get data, but I cannot get it to work (by guessing, seems undocumented)

In [176]:
# start with one dataset at MetaboLights, TSQ data described in Longnecker et al. 2024 (only other dataset ready is the untargeted data)
study_id = 'MTBLS2356'

In [177]:
#while testing, if the FTP command fails the connection is left open and the next command gives error
#error is: AttributeError: 'NoneType' object has no attribute 'sendall'
ftp = FTP('ftp.ebi.ac.uk') #address from MetaboLights webpage
ftp.login()
ftpDataAddress = '/pub/databases/metabolights/studies/public/' + study_id
ftp.cwd(ftpDataAddress)
#ftp.retrlines('LIST') #this will only print to console, not what I want
fileList = ftp.nlst() #can use this to make a list that will be searchable
#fileList

In [178]:
#start with the metadata about the samples so I can convert each sample to time/lat/lon/depth to match the CMAP requirements
str = 's_' + study_id #this is the search string for the data files
metadataFiles = [v for v in fileList if str in v] 
metadataFiles = pd.DataFrame(metadataFiles,columns = ['files'])
readFile = metadataFiles.loc[0,'files']

# metadataFiles: put them here 
# Is there a way to download an FTP file and not write it disk?
writeFile = 'data/' + 'tempMetadata.txt'

with open(writeFile,'wb') as fp:
    try:
        retr_command = f"RETR {readFile}"
        ftp.retrbinary(retr_command, fp.write)
    except Exception as e: 
        print(f"Error during quit: {e}")
    except AttributeError as e: 
        print(f"AttributeError during quit: {e} - connection was likely already closed.")

# now read in the result
metadata_aboutSamples = pd.read_table(writeFile,delimiter = '\t')

In [179]:
# Now get the data files (more than one because things are split positive/negative ion mode...concatenate them later
str = 'm_' + study_id #this is the search string for the data files
dataFiles = [v for v in fileList if str in v] #Python syntax, will make a list
dataFiles = pd.DataFrame(dataFiles,columns = ['files']) #I find the dataframe easier to manage than the list
tsvFile = pd.DataFrame(); #this will be the data file

# readDataFile = dataFiles.loc[0,'files']
#have to do some concatenating here bc positive and negative ion mode data
for idx in range(len(dataFiles)):
    readDataFile = dataFiles.loc[idx,'files']
    writeDataFile = 'data/' + 'tempData.tsv'
          
    with open(writeDataFile,'wb') as fp:
        #try-except to make sure the FTP closes
        try:
            retr_command = f"RETR {readDataFile}"
            #pdb.set_trace()
            ftp.retrbinary(retr_command, fp.write)
        except Exception as e: 
            print(f"Error during quit: {e}")
    
    #read in the temporary file and add to tsvFile file
    tsvFile = pd.concat([tsvFile,pd.read_table(writeDataFile,delimiter = '\t')],ignore_index=True) #append is no longer valid

In [180]:
#finally, details about the experiment are easy because the filename is generic
#information about the project is in the i_Investigation file
writeFile = 'data/' + 'i_Investigation.txt'
readFile = 'i_Investigation.txt'

with open(writeFile,'wb') as fp:
    #try-except to make sure the FTP closes
    try:
        retr_command = f"RETR {readFile}"
        ftp.retrbinary(retr_command, fp.write)
    except Exception as e: 
        print(f"Error during quit: {e}")

# del writeFile, readFile

#open up the txt file with the experiment data
writeFile = 'data/i_Investigation.txt'  
with open(writeFile, 'r') as f:
    metadata_aboutExperiment = f.read()

In [181]:
#this might not be necessary, but leave here in case I go back to it.
# files = {}
# files['data'] = tsvFile #this also holds the metadata_variables
# files['metadata_project'] = metadata_aboutExperiment
# files['metadata_samples'] = metadata_aboutSamples

In [182]:
ftp.quit()  #close the FTP connection

'221 Goodbye.'

In [183]:
# pull what I can from the sample information at MetaboLights
#pull 'Source Name' as I need that later to match to columns in the data file
sampleNames  = metadata_aboutSamples['Source Name']
depth = metadata_aboutSamples['Factor Value[Depth]']
#time is messier and the MetaboLights columns names are long, so shorten them to make this easier
temp = metadata_aboutSamples[['Factor Value[Sampling year date]','Factor Value[Sampling month date]',
                 'Factor Value[Sampling day date]','Factor Value[Hour of the day]','Factor Value[Minute of the hour]']]
temp.columns = ['year','month','day','hour','minute']

step1 = pd.to_datetime(dict(year=temp.year,month=temp.month,day = temp.day,hour = temp.hour,minute=temp.minute))
date_cmap = step1.dt.strftime("%Y-%m-%dT%H:%M:%S")

#### Need BIOS-SCOPE discrete file for lat/lon information

In [184]:
#will need the BIOS-SCOPE discrete data file for station information - that will have both BATS and BIOS-SCOPE data in it
#this is slow, so leave as its own cell
fName = 'data/BATS_BS_COMBINED_MASTER_latest.xlsx';
BSdata = pd.read_excel(open(fName,'rb'),sheet_name = 'DATA')

In [185]:
#then setup the ability to use the information in BSdata by mathing sample names:

#MetaboLights required samples to begin with a letter, I used 's' and need to strip that out 
NewID_inMTBLS  = pd.to_numeric(sampleNames.str.strip('s')) 
#convert the series into a dataframe:
s_df = NewID_inMTBLS.reset_index()

#use merge as it will be sorted in the right order; only keep rows with data found in MetaboLights
merged_df = pd.merge(BSdata,s_df,how='right',left_on='New_ID',right_on='Source Name')

#get the list of cruise names and find the unique ones - this will go on the sheet with metadata about the project
unCruises = pd.DataFrame(merged_df.loc[:,'Cruise_ID'].unique())
unCruises.columns = ['cruise_names']

### Operate on the files I collected via FTP
Remember data are in the tsvFile

In [186]:
tsvFile.columns

Index(['database_identifier', 'chemical_formula', 'smiles', 'inchi',
       'metabolite_identification', 'mass_to_charge', 'fragmentation',
       'modifications', 'charge', 'retention_time',
       ...
       's1035700708', 's1035700712', 's1035700704', 's1035700718',
       's1035802401', 's1035802424', 's1035802408', 's1035802412',
       's1035802404', 's1035802418'],
      dtype='object', length=393)

In [187]:
#column with metabolite name is  (database identifier would be more generic, need to talk to CMAP people about this)
mtabColumn = 'database_identifier'
# mtabColumn = 'metabolite_identification' 

#only keep the columns that are in sampleNames
dataColumns = tsvFile.columns[tsvFile.columns.isin(sampleNames)]
dataOnly = tsvFile.loc[:,dataColumns].transpose() #index is the 's' numbered samples

dataOnly.columns = tsvFile[mtabColumn] #label the columns with the metabolite information, will also use this for the sheet with metadata about the variables
nVariables = len(dataOnly.columns) #need this for the sheet for the metadata on the variables

In [188]:
'''
Assemble the data into the CMAP format
'''
# Required variables are time, lat, lon, depth
df = pd.DataFrame(columns=['time','lat','lon','depth'])
df['time'] = date_cmap.to_frame()
df['depth'] = depth.to_frame()
df['lat'] = merged_df['latN'].to_frame()
df['lon'] = -merged_df['lonW'].to_frame() #need negative number to put this into -180 to 180 space
#df.insert(1,'test',merged_df['New_ID']) #check that I have the indexing right
#df.insert(1,'test2',s_df['Source Name'])
df.insert(1,'forIndex',sampleNames) #need an index to keep the rows matched up
df.set_index('forIndex',inplace=True)

#concatenate with the data in dataOnly
df = pd.concat([df,dataOnly],axis=1,ignore_index = False)

In [189]:
'''
Work on the second sheet: metadata about the variables; use the CMAP dataset template to setup the dataframe so I get the column headers right
'''
fName = 'datasetTemplate.xlsx' #downloaded from CMAP website
sheet_name = 'vars_meta_data'
vars = pd.read_excel(fName, sheet_name=sheet_name)
cols = vars.columns.tolist()
#df2 will be the dataframe with the metadata about the variables, set it up empty here
df2 = pd.DataFrame(columns=cols,index = pd.RangeIndex(0,nVariables,1))

In [190]:
'''
    Define a funtion to use the the chebi tool to find synonyms. Krista Longnecker, 12 July 2025
    input: oneName is a string with the CHEBI number (just the number, no prefix)
'''
def getSynonym(oneName):
    possibles = ChebiEntity(oneName).get_names()
    justNames = []
    for idx in range(len(possibles)):
        justNames.append(possibles[idx].get_name())
    
    return justNames

In [191]:
#need details (metadata) about the metabolites 

# this is only a partial list of variables for the moment
df2['var_short_name'] = dataOnly.columns
df2.loc[:,'var_long_name'] = tsvFile.loc[:,'metabolite_identification']
df2.loc[:,'var_sensor'] = 'Triple quadrupole mass spectrometer (TSQ Vantage, Thermo Scientific)'
df2.loc[:,'var_unit'] = 'pM' #this is in the protocols, but I also have some inside information here
df2.loc[:,('var_spatial_res')] = 'irregular'
df2.loc[:, ('var_temporal_res')] = 'irregular'
df2.loc[:,('var_discipline')] = 'chemistry'
df2.loc[:,('visualize')] = 1 #yes/no, all metabolites can be visualized

In [192]:
#use unCruises to get the list of cruises, put that at the beginning, then add text that will work for the whole project
misc_keywords = ','.join(unCruises['cruise_names']) + ', Bermuda Institute of Ocean Sciences, bio, biogeo, biogeochemistry, biology, BIOS, Bottle, cruise, Discrete, in situ, insitu, in-situ, North Atlantic Ocean, observation, '
#then need to add the string with the names for each metabolite...do that in the next loop

In [193]:
#go through one metabolite at a time and make var_keywords
#'var_keywords' will be the hardest as the metabolites have many, many keywords. 
for idx,item in df2.iterrows():
    justNames = ', '.join(getSynonym(df2.loc[idx,'var_short_name'].strip('CHEBI:')))
    df2.loc[idx,'var_keywords'] = misc_keywords + justNames
    #print(df2.loc[idx,'var_short_name'])
    del justNames    

In [194]:
#make a function to pull details from MetaboLights i file
#this goes through one at a time...could alter to get all in one function (get stuff, and clean stuff as two steps)
def getMetaboLights(pattern,text):
    #pattern = r'Study Identifier(.*)' # Captures anything after "Study Identifier = "
    extracted_data = re.findall(pattern, text) #this is a list
    extracted_data = ' '.join(extracted_data) # ? really, this seems odd, but works.
    
    #tidy up the string
    original_string = extracted_data
    chars_to_remove = ['<p>', '</p>','\t']
    
    # Using regular expressions (for more complex patterns or multiple occurrences)
    pattern = "|".join(map(re.escape, chars_to_remove)) # Escapes special characters for regex
    oneVar = re.sub(pattern, "", original_string)
    return oneVar

In [195]:
'''
Gather the details I need from the metadata_aboutExperiment
'''
project_description = getMetaboLights(r'Study Description(.*)',metadata_aboutExperiment)
dataset_short_name = getMetaboLights(r'Study Identifier(.*)',metadata_aboutExperiment)
dataset_long_name = getMetaboLights(r'Study Title(.*)',metadata_aboutExperiment)


In [196]:
# gather up the dataset_meta_data into df3

df3 = pd.DataFrame({
    'dataset_short_name': [dataset_short_name],
    'dataset_long_name': [dataset_long_name],
    'dataset_version': ['1.0'],
    'dataset_release_date': ['2025-06-25'],
    'dataset_make': ['observation'],
    'dataset_source': ['Elizabeth Kujawinski, Woods Hole Oceanographic Institution'],
    'dataset_distributor': ['Elizabeth Kujawinski, Woods Hole Oceanographic Institution'],
    'dataset_acknowledgement': ['We thank the BIOS-SCOPE project team and the BATS team for assistance with sample collection, processing, and analysis. The efforts of the captains, crew, and marine technicians of the R/V Atlantic Explorer are a key aspect of the success of this project. This work supported by funding from the Simons Foundation International.'],
    'dataset_history': [''],
    'dataset_description': [project_description],
    'dataset_references': ['https://www.ebi.ac.uk/metabolights/editor/' + study_id],
    'climatology': [0]
    })


#insert the list of unique cruises from the data pulled from the BIOS-SCOPE discrete file
df3 = pd.concat([df3,unCruises],axis=1,ignore_index = True)

In [197]:
fName_CMAP = 'data/forCMAPfromMetabolights.xlsx'
dataset_names = {'data': df, 'dataset_meta_data': df3, 'vars_meta_data': df2}
with pd.ExcelWriter(fName_CMAP) as writer:
    for sheet_name, data in dataset_names.items():
        data.to_excel(writer, sheet_name=sheet_name, index=False)

In [198]:
raise UserWarning('Stopping and leave code below for historical reasons, code will not run')

UserWarning: Stopping and leave code below for historical reasons, code will not run

In [ ]:
#If I can get isatools to install, this will be an easier way (I think/hope) to access data at BCO-DMO. There is also an R equivalent at metabolighteR that provides access to MetaboLights REST API

In [ ]:
#pip install isatools #fails with this error:AttributeError: module 'pkgutil' has no attribute 'ImpImporter'. Did you mean: 'zipimporter'?
pip install git+https://github.com/ISA-tools/isa-api/

In [ ]:
import isatools

In [ ]:
from isatools.net import mtbls as MTBLS
tmp_dir = MTBLS.get('MTBLS2356')

In [ ]:
""" Create a Frictionless Data Package """

biosscope = Package(name='biosscope-bcodmo-datasets', profile='data-package')
biosscope.title = 'Bermuda Institute of Ocean Sciences Simons Collaboration on Ocean Processes and Ecology'
biosscope.description = 'BIOSSCOPE datasets from BCO-DMO'
biosscope.created = rfc3339_datetime_str()
biosscope.sources = []

for index, dataset in metadata.iterrows():

  # Save the dataset as a 'source' in the Package
  source = {
    'path': dataset['dataset'],
    'title': dataset['title'],
    'doi': dataset['doi']
  }
  biosscope.sources.append(source)


  # Get the BCO-DMO parameters
  parameters = get_sparql_dataframe(SPARQL_ENDPOINT, DATASET_PARAMS_QUERY.replace('{dataset_uri}', dataset['dataset']))
  schema = []
  for param_index, parameter in parameters.iterrows():
    param = {}
    param['bcodmo:name'] = parameter['supplied_name']
    if parameter['supplied_definition'] is not None:
      param['bcodmo:description'] = parameter['supplied_definition']
    if parameter['datatype'] is not None:
      param['bcodmo:datatype'] = parameter['datatype']
    if parameter['units'] is not None:
      param['bcodmo:units'] = parameter['units']
    if parameter['format'] is not None:
      param['bcodmo:valueFormat'] = parameter['format']
    schema.append(param)

  # Get the 'data' files for a Dataset (skip any supplemental documentation)
  files = get_sparql_dataframe(SPARQL_ENDPOINT, DATASET_FILES_QUERY.replace('{dataset_uri}', dataset['dataset']))
  for file_index, file in files.iterrows():

    # Use Frictionless to describe the file
    datafile = describe(file['url'])

    # Get Table stats
    if datafile.type == 'table':
      datafile.infer(stats=True)

    # Specify which dataset this file belongs to
    datafile.sources = [source]

    # If the file is marked as the primary file for the dataset, attach the parameters to the file
    if schema is not None and file['is_primary_data_file'] == 'true':
      datafile.custom['bcodmo:parameters'] = schema

    # Add the file to the package
    biosscope.add_resource(datafile)

# Save the package
print(biosscope.to_json())
biosscope.to_json('datapackage.json')